https://nbviewer.jupyter.org/github/AdamSwenson/CanvasHacks/blob/master/Notebooks/Credit%20only%20assignment%20tools.ipynb?flush_cache=true

# Get started

Click the icon with 3 rings in the upper right (the tooltip says 'Execute on Binder') to create your own mini-server to run this notebook.

After you've opened the notebook in binder, click the run button above a few times. Each click runs the code in the highlighted cell. You can also press shift+return to run it.

Try it with the next cell:

In [ ]:
print("You've successfully run this cell!")



You'll be able to tell that a cell is running by an asterisk appearing in the brackets to the left ( [ * ]). Some tasks may take a couple of minutes to run, so it may appear frozen.

The next 2 cells will grab and import all the code that operates behind the scenes. (If you're curious, you can check it out here: https://github.com/AdamSwenson/CanvasHacks)


In [ ]:
# Install our library to the server
!pip install CanvasHacks

In [ ]:
%cd ../

from CanvasHacks import environment
from CanvasHacks.Configuration import InteractiveConfiguration
from CanvasHacks.Repositories.DataManagement import DataStore

from CanvasHacks.RequestTools import get_all_course_assignments, get_assignments_needing_grading
import CanvasHacks.GradingTools as GT
import CanvasHacks.DownloadProcessingTools as PT
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.CredentialFields import make_credentials_input
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser, view_selected_assignments, view_ungraded_assignments

# Download students
from CanvasHacks.Repositories.StudentData import StudentRepository


make_credentials_input()

## Canvas url

Finally, we need the url of your campus canvas installation. When we make requests to the canvas server, we will be using this in our requests. For details, see the Canvas API documentation: https://canvas.instructure.com/doc/api/index.html

For CSUN users, it should be: 
    
    https://canvas.csun.edu/api/v1/courses

For non-csun users, make sure you provide the equivalent up to the '/courses'

Just paste the url in the box, it will be read when needed

Do not add a slash at the end after '/courses'! 

In [ ]:
make_canvas_url_input()

# Figure out what needs grading

Run the cell below. It will display buttons for each ungraded assignment where at least one student has submitted the assignment. 

Use these buttons to select which assignments to grade. Unselected assignments are blue; selected are green.

In [ ]:
make_assignment_chooser()

# Download and process student submissions

In [ ]:
results = []

for course_id in environment.CONFIG.course_ids:
    print('course', course_id)
    for assignment_id, name in environment.CONFIG.assignments:
        store = DataStore(assignment_id=assignment_id, assignment_name=name, course_id=course_id)

        print("Processing {}".format(name))
        # make folder
        # download student submissions 
        response = PT.get_submissions(course_id, assignment_id)
        print("{} responses received".format(len(response)))
        store.submissions = PT.process_response_without_saving_files(response)
        # give credit for non-empty submissions
        c = GT.determine_credit(store.submissions)
        # stow the results in our data object
        store.credit = c['credit']
        store.no_credit = c['nocredit']
        store.print_counts()
        # Add the now full-of-data object to our results list
        results.append(store)

# Load student names etc
studentRepo = StudentRepository(environment.CONFIG.course_ids)

# Read student work and check that properly categorized

In [ ]:
from ipywidgets import widgets
from IPython.display import display

def make_assignment_header(store):
    """Displays the header for each assignment"""
    entry = """
    <h1>{}</h1>
    <h2>Class id: {}</h2>"""
    return display(widgets.HTML(entry.format(store.assignment_name, store.course_id)))


def make_text_display(student_id, text):
    """Handles the formatting of the student's text"""
    entry = """
          <div id='%s'>
            <h3>%s</h3>
            <p>
                %s
            </p>
        </div>"""
    e = entry % (student_id, student_id, text)
    return widgets.HTML(e)

def make_submission_output(text, student_id, credit_list):
    """Creates the display of the submitted text with a toggle button to 
    update whether the student receives credit 
    """
    bval = 'Credit' if student_id in credit_list else 'No credit'
    credit_button = widgets.ToggleButtons(
        options=['Credit', 'No credit'],
        value=bval,
        description='',
        disabled=False
    )
    
    ta = make_text_display(student_id, text)

    def on_value_change(change):
        """The relevant callback
        NB the use of scope to define student_id"""
        v = change['new']
        print(v, student_id)
        try:
            if v == 'Credit':
                credit_list.append(student_id)
            elif v == 'No credit':
                credit_list.remove(student_id)
        except ValueError:
            pass

    credit_button.observe(on_value_change, names='value')
    display(widgets.VBox([ta, credit_button]))

def make_consolidated_text_fields(store):
    """Displays each entry with a toggle to adjust whether the 
    student receives credit"""
    for r in store.submissions:
        make_submission_output(r['body'], r['student_id'], store.credit)

In [ ]:
for s in results:
    make_assignment_header(s)
    make_consolidated_text_fields(s, studentRepo)

# Upload the grades to canvas

In [ ]:
from CanvasHacks.UploadGradeTools import upload_students_receiving_credit
from ipywidgets import widgets
from IPython.display import display

def make_upload_button(store):
    def upload_callback(event):
        # change button style to indicate working
        b.button_style='warning'
        # upload grades for students receiving credit
        upload_students_receiving_credit(store)
        # change button style
        b.button_style='success'
        
    desc = 'Upload grades for {}'.format(store.assignment_name)
    layout= widgets.Layout(width='50%')
    b = widgets.Button(description=desc, button_style='danger', layout=layout)
    b.on_click(upload_callback)
    
    display(b)

In [ ]:
for store in results:
    make_upload_button(store)